In [1]:
import yaml
import importlib

In [2]:
# Path to your YAML configuration file
config_file_path = r'D:\tracks_interactions\config.yaml'

In [140]:
# Reading the YAML file
with open(config_file_path, 'r') as file:
    config = yaml.safe_load(file)



In [29]:
req_graphs = [signal for x in config['graphs'] for signal in x['signals']]
req_graphs

['ch0_nuc', 'ch0_cyto', 'area']

In [7]:
req_custom_functions = [x for x in config['cell_measurements'] if not x['source']=='regionprops' and not x['source']=='track_gardener']
for f in req_custom_functions:
    try:
        module = importlib.import_module(f['source'])
        func = getattr(module, f['function'])
    except ImportError:
        print(f'Custom module "{f["source"]}" could not be imported.')
    except AttributeError:
        print(f'Custom function "{f["function"]}" could not be found in {f["source"]}.')
    except:
        print('shit')

Custom module "D:\test_data\custom_measurements.py" could not be imported.


In [11]:
f['function']

'median_intensity'

In [16]:
import os

file_path = f['source']
function_name = f['function']

if not os.path.exists(file_path):
    print(f"File '{file_path}' does not exist.")

# Load the module from the specified file path
module_name = os.path.splitext(os.path.basename(file_path))[0]  # Extract module name from file
spec = importlib.util.spec_from_file_location(module_name, file_path)
module = importlib.util.module_from_spec(spec)
try:
    # Execute the module to load it
    spec.loader.exec_module(module)
    # Get the function
    func = getattr(module, function_name)
except (FileNotFoundError, AttributeError) as e:
    print(f"Function '{function_name}' could not be loaded from '{file_path}': {e}")

In [110]:
%load_ext autoreload
%autoreload 2

from tracks_interactions.db.config_functions import load_function_from_path, testConfigFile, check_unique_names
import tracks_interactions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
sys.path.append('..')

In [33]:
config['cell_measurements']

[{'function': 'area', 'source': 'regionprops'},
 {'function': 'intensity_mean',
  'name': 'nuc',
  'source': 'regionprops',
  'channels': ['ch0', 'ch1']},
 {'function': 'ring_intensity',
  'name': 'cyto',
  'source': 'track_gardener',
  'channels': ['ch0', 'ch1'],
  'thickness': 5},
 {'function': 'median_intensity',
  'source': 'D:\\test_data\\custom_measurements.py',
  'channels': ['ch0', 'ch1']}]

In [265]:
import numpy as np
import importlib
from skimage.measure import regionprops
from tracks_interactions.db.config_functions import load_function_from_path, load_function_from_module

# Reading the YAML file
with open(config_file_path, 'r') as file:
    config = yaml.safe_load(file)


def create_calculate_signals_function(config):
    """
    Function to generate for signal calculation based on the requirements in the configuration file.
    input:
        config: dictionary containing all configuration information
    output:
        calculate_cell_signals: function that calculates all signals for a given cell
    """

    ch_list = [x.get('name') for x in config['signal_channels']]

    # regionprops with no signal
    reg_no_signal = [x['function'] for x in config['cell_measurements'] if x['source']=='regionprops' and not 'channels' in x]

    # regionprops with signal
    reg_signal = [x for x in config['cell_measurements'] if x['source']=='regionprops' and 'channels' in x]

    # track gardener implemented functions
    gardener_signal = [x for x in config['cell_measurements'] if x['source']=='track_gardener']

    # custom functions
    custom_signal = [x for x in config['cell_measurements'] if not x['source']=='track_gardener' and not x['source']=='regionprops']

    #######################################################################################################################
    def calculate_cell_signals(cell,t,ch_data_list):
        """
        Function to calculate signals for every given cell.
        input:
            cell: cell object from regionprops
            ch_data_list: list of all channel data
        output:
            cell_dict: dictionary containing all measurements for the cell
        """

        # create an empty dictionary
        cell_dict = dict()

        #######################################################################################################################
        # add all measurements directly from regionprops
        for m in reg_no_signal:
            cell_dict[m] = cell[m]

        #######################################################################################################################
        # add all measurements from regionprops with channels
        if len(reg_signal) > 0:

            signal_cube = np.zeros((cell.bbox[2]-cell.bbox[0],cell.bbox[3]-cell.bbox[1],len(ch_list)),dtype=ch_data_list[0].dtype)
            for ind,ch in enumerate(ch_data_list):
                if ch.ndim == 3:
                    cell_signal = ch[t,cell.bbox[0]:cell.bbox[2],cell.bbox[1]:cell.bbox[3]]
                if ch.ndim == 2:
                    cell_signal = ch[cell.bbox[0]:cell.bbox[2],cell.bbox[1]:cell.bbox[3]]

                signal_cube[:,:,ind] = cell_signal

            result = regionprops(cell.image.astype(int),intensity_image=signal_cube)

            for m in reg_signal:
                for ch in m['channels']:
                    cell_dict[ch + '_' + m['name']] = result[0][m['function']][ch_list.index(ch)]
      

        #######################################################################################################################
        # add measurements from the track gardener
        # for simplicity we calculate for all the channels - may be revisited later
        if len(gardener_signal) > 0:
            for m in gardener_signal:
                f = load_function_from_module('tracks_interactions.db.db_functions', m['function'])
                result = f(cell,t,ch_data_list,kwargs = m)
                for ch in m['channels']:
                    cell_dict[ch + '_' + m['name']] = result[ch_list.index(ch)]


        #######################################################################################################################
        # add measurements from the custom functions
        if len(custom_signal) > 0:
            for m in custom_signal:
                f = load_function_from_path(m['source'], m['function'])
                result = f(cell,t,ch_data_list,kwargs = m)
                for ch in m['channels']:
                    cell_dict[ch + '_' + m['name']] = result[ch_list.index(ch)]

        return cell_dict

    return calculate_cell_signals


In [266]:
calculate_cell_signals = create_calculate_signals_function(config)

In [267]:
import numpy as np
from skimage.measure import regionprops

n=100

t = np.zeros([n,n]).astype(int)
t[50:60,50:60]=1
res = regionprops(t)
res

In [268]:
%time
rs = calculate_cell_signals(res[0],1,[np.ones([2,n,n]),np.ones([2,n,n])*2])
rs

CPU times: total: 0 ns
Wall time: 0 ns


{'area': 100.0,
 'ch0_nuc': 1.0,
 'ch1_nuc': 2.0,
 'ch0_cyto': array([[False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ True, False, False, False, False, False, False, False, False,
         False, False,  True],
        [ T

In [269]:
import napari
viewer = napari.Viewer()
viewer.add_image(rs['ch0_cyto'])

<Image layer 'Image' at 0x1892b3c7cd0>

In [263]:
config

{'experiment_settings': {'experiment_name': '2023_E6_exp',
  'description': 'Cisplatin treated ovarian cancer cells. ch0 - DHB, ch1 - 53BP1'},
 'signal_channels': [{'name': 'ch0',
   'path': 'D:\\test_data\\E6_C0_level_1.zarr',
   'lut': 'green'},
  {'name': 'ch1', 'path': 'D:\\test_data\\E6_C1_level_1.zarr', 'lut': 'red'}],
 'cell_measurements': [{'function': 'area', 'source': 'regionprops'},
  {'function': 'intensity_mean',
   'name': 'nuc',
   'source': 'regionprops',
   'channels': ['ch0', 'ch1']},
  {'function': 'ring_intensity',
   'name': 'cyto',
   'source': 'track_gardener',
   'channels': ['ch0', 'ch1'],
   'thickness': 1}],
 'database': {'path': 'D:\\test_data\\Exp6_gardener_v6.db'},
 'graphs': [{'name': 'DHB',
   'signals': ['ch0_nuc', 'ch0_cyto'],
   'colors': ['yellow', 'green']},
  {'name': 'ar', 'signals': ['area'], 'colors': ['yellow']}],
 'cell_tags': {'modified': 'x',
  'apoptosis': 'A',
  'mitosis': 'M',
  'disappear': 'D',
  'test': 'T'}}